# Entendiendo Tipos de datos en Python

La efectiva ciencia basada en datos y la computación requieren comprender cómo se almacenan y manipulan los datos.Esta sección describe y contrasta cómo se manejan las matrices de datos en el lenguaje Python y cómo NumPy mejora esto.

Los usuarios de Python a menudo se sienten atraídos por su facilidad de uso, una de las cuales es la escritura dinámica.Mientras que un lenguaje de tipo estático como C o Java requiere que cada variable se declare explícitamente, un lenguaje de tipo dinámico como Python omite esta especificación. Por ejemplo, en C puede especificar una operación particular de la siguiente manera:

```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

Mientras que en Python, la operación equivalente podría escribirse de esta manera:

```python
# Python code
result = 0
for i in range(100):
    result += i
```

Observe la diferencia principal: en C, los tipos de datos de cada variable se declaran explícitamente, mientras que en Python los tipos se infieren dinámicamente. Esto significa, por ejemplo, que podemos asignar cualquier tipo de datos a cualquier variable:

```python
# Python code
x = 4
x = "four"
```

Aquí hemos cambiado el contenido de `` x`` de un entero a una cadena. Lo mismo en C llevaría (dependiendo de la configuración del compilador) a un error de compilación u otras consecuencias no intencionadas:

```C
/* C code */
int x = 4;
x = "four";  // FAILS
```

Este tipo de flexibilidad es una pieza que hace que Python y otros lenguajes de tipo dinámico sean convenientes y fáciles de usar.
Comprender *cómo* esto funciona es una pieza importante de aprendizaje para analizar datos de manera eficiente y efectiva con Python.
Pero lo que también señala esta flexibilidad de tipo es el hecho de que las variables de Python son más que solo su valor; También contienen información adicional sobre el tipo de valor. Exploraremos esto más en las secciones que siguen.

## Un Python Integer Es Más Que Solo un Integer

La implementación estándar de Python está escrita en C.
Esto significa que cada objeto de Python es simplemente una estructura C ingeniosamente disfrazada, que contiene no solo su valor, sino también otra información. Por ejemplo, cuando definimos un número entero en Python, como `` x = 10000 ``, `` x`` no es solo un número entero "en bruto". En realidad, es un puntero a una estructura compuesta de C, que contiene varios valores.
Mirando a través del código fuente de Python 3.4, encontramos que la definición de tipo entero (largo) efectivamente se ve así (una vez que se expanden las macros C):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

Un entero en Python 3.4 en realidad contiene cuatro piezas:

- ``ob_refcnt``, a reference count that helps Python silently handle memory allocation and deallocation
- ``ob_type``, which encodes the type of the variable
- ``ob_size``, which specifies the size of the following data members
- ``ob_digit``, which contains the actual integer value that we expect the Python variable to represent.

Esto significa que hay algo de sobrecarga al almacenar un número entero en Python en comparación con un número entero en un lenguaje compilado como C, como se ilustra en la siguiente figura:

![Integer Memory Layout](figures/cint_vs_pyint.png)

Aquí `` PyObject_HEAD`` es la parte de la estructura que contiene el recuento de referencia, el código de tipo y otras piezas mencionadas anteriormente.

Observe la diferencia aquí: un entero C es esencialmente una etiqueta para una posición en la memoria cuyos bytes codifican un valor entero.
Un entero de Python es un puntero a una posición en la memoria que contiene toda la información del objeto de Python, incluidos los bytes que contienen el valor entero.
Esta información adicional en la estructura entera de Python es lo que permite que Python se codifique de manera tan libre y dinámica.
Sin embargo, toda esta información adicional en los tipos de Python tiene un costo, que se hace especialmente evidente en las estructuras que combinan muchos de estos objetos.

## Una Python List Es Más Que Solo una List

Consideremos ahora qué sucede cuando usamos una estructura de datos de Python que contiene muchos objetos Python.
El contenedor estándar de elementos múltiples mutables en Python es la lista. Podemos crear una lista de enteros de la siguiente manera:

In [ ]:
L = list(range(10))
L

In [ ]:
type(L[0])

O, similarmente, una lista de strings:

In [ ]:
L2 = [str(c) for c in L]
L2

In [ ]:
type(L2[0])

Debido a la tipificación dinámica de Python, incluso podemos crear listas heterogéneas

In [ ]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

Pero esta flexibilidad tiene un costo: para permitir estos tipos flexibles, cada elemento de la lista debe contener su propia información de tipo, recuento de referencias y otra información, es decir, cada elemento es un objeto completo de Python. En el caso especial de que todas las variables sean del mismo tipo, gran parte de esta información es redundante: puede ser mucho más eficiente almacenar datos en una matriz de tipo fijo. La diferencia entre una lista de tipo dinámico y una matriz de tipo fijo (estilo NumPy) se ilustra en la siguiente figura:

![Array Memory Layout](figures/array_vs_list.png)

En el nivel de implementación, la matriz contiene esencialmente un puntero único a un bloque contiguo de datos.
La lista de Python, por otro lado, contiene un puntero a un bloque de punteros, cada uno de los cuales a su vez apunta a un objeto completo de Python como el entero de Python que vimos anteriormente.
Una vez más, la ventaja de la lista es la flexibilidad: dado que cada elemento de la lista es una estructura completa que contiene tanto datos como información de tipo, la lista puede llenarse con datos de cualquier tipo deseado.
Las matrices de tipo fijo tipo NumPy carecen de esta flexibilidad, pero son mucho más eficientes para almacenar y manipular datos.

## Matrices de tipo fijo en Python

Python ofrece varias opciones diferentes para almacenar datos en búferes de datos eficientes de tipo fijo.
El módulo incorporado `` array `` (disponible desde Python 3.3) se puede utilizar para crear matrices densas de un tipo uniforme:

import array
L = list(range(10))
A = array.array('i', L)
A

Aquí `` 'i' `` es un código de tipo que indica que los contenidos son enteros.

Sin embargo, mucho más útil es el objeto `` ndarray`` del paquete NumPy.
Si bien el objeto `` array `` de Python proporciona un almacenamiento eficiente de datos basados en matrices, NumPy se suma a estas *operaciones* eficientes en esos datos.

Comenzaremos con la importación estándar de NumPy, bajo el alias `` np``:

In [ ]:
import numpy as np

## Crear matrices a partir de listas de Python

Primero, podemos usar `` np.array`` para crear matrices a partir de listas de Python:

In [ ]:
# integer array:
np.array([1, 4, 2, 5, 3])

Recuerde que, a diferencia de las listas de Python, NumPy está restringido a matrices que contienen el mismo tipo.
Si los tipos no coinciden, NumPy se elevará si es posible (aquí, los enteros se elevan a coma flotante):

In [ ]:
np.array([3.14, 4, 2, 3])

Si queremos establecer explícitamente el tipo de datos de la matriz resultante, podemos usar la palabra clave `` dtype``:

In [ ]:
np.array?

In [ ]:
np.array([1, 2, 3, 4], dtype='float32')

Finalmente, a diferencia de las listas de Python, las matrices NumPy pueden ser explícitamente multidimensionales; Aquí hay una forma de inicializar una matriz multidimensional utilizando una lista de listas:

In [ ]:
# nested lists result in multi-dimensional arrays
np.array([range(i, i + 3) for i in [2, 4, 6]])

Las listas internas se tratan como filas de la matriz bidimensional resultante.

## Crear matrices desde cero

Especialmente para matrices más grandes, es más eficiente crear matrices desde cero utilizando rutinas integradas en NumPy.
Aquí hay varios ejemplos:

In [ ]:
# Create a length-10 integer array filled with zeros
np.zeros(10, dtype=int)

In [ ]:
# Create a 3x5 floating-point array filled with ones
np.ones((3, 5), dtype=float)

In [ ]:
# Create a 3x5 array filled with 3.14
np.full((3, 5), 3.14)

In [ ]:
# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range() function)
np.arange(0, 20, 2)

In [ ]:
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)

In [ ]:
np.random.random?

In [ ]:
# Create a 3x3 array of uniformly distributed
# random values between 0 and 1
np.random.random((3, 3))

In [ ]:
# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

In [ ]:
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))

In [ ]:
# Create a 3x3 identity matrix
np.eye(3)

In [ ]:
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

## NumPy Standard Data Types

Las matrices NumPy contienen valores de un solo tipo, por lo que es importante tener un conocimiento detallado de esos tipos y sus limitaciones.
Debido a que NumPy está construido en C, los tipos serán familiares para los usuarios de C, Fortran y otros lenguajes relacionados.

Los tipos de datos estándar de NumPy se enumeran en la siguiente tabla.
Tenga en cuenta que al construir una matriz, se pueden especificar utilizando una cadena:

```python
np.zeros(10, dtype='int16')
```

O usando el asociado objeto NumPy:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) stored as a byte |
| ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
| ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
| ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Shorthand for ``float64``.| 
| ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| ``complex_``  | Shorthand for ``complex128``.| 
| ``complex64`` | Complex number, represented by two 32-bit floats| 
| ``complex128``| Complex number, represented by two 64-bit floats| 